<a href="https://colab.research.google.com/github/arzuunr/100_day_challenge/blob/main/Aterimlerini%C3%87ekme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Hedef URL
url = "https://www.investopedia.com/terms-beginning-with-a-4769351"

# Headers
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd"
}

# Sayfayı çek
response = requests.get(url, headers=headers)

# Hata kontrolü
if response.status_code == 200:
    # HTML'yi parse et
    soup = BeautifulSoup(response.content, "html.parser")

    # Terimlerin bulunduğu HTML elementlerini bul (class'ı doğrulayın)
    terms = []
    for link in soup.find_all("a"):  # Önce tüm <a> etiketlerini dene
        term = link.text.strip()
        if term and len(term.split()) > 0:  # Boş veya anlamsız içerikleri filtrele
            terms.append(term)

    # Benzersiz terimleri al
    terms = list(dict.fromkeys(terms))  # Tekrarları kaldır

    # Terimleri bir DataFrame'e kaydet
    df = pd.DataFrame(terms, columns=["Term"])
    df.to_csv("investopedia_terms_a.csv", index=False)

    print(f"{len(terms)} terim çekildi. Örnek: {terms[:5]}")
else:
    print(f"Hata: {response.status_code} - {response.text}")

# Rate limiting için bekleme
time.sleep(2)

395 terim çekildi. Örnek: ['Skip to content', 'News', 'Markets', 'Companies', 'Earnings']


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

url = "https://www.odeabank.com.tr/hakkimizda/oblog/adan-zye-bankacilik-ve-finans-terimleri-sozlugu"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.4 Safari/605.1.15",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "tr-TR,tr;q=0.9,en-GB,en;q=0.9"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

terms = []
# "A" başlıklı bölümü bul
a_section = soup.find("h2", string="A")  # "A" başlığını bul
if a_section:
    # Başlıktan sonraki terimleri al (örneğin, bir <ul> veya <div> içinde)
    term_container = a_section.find_next("ul") or a_section.find_next("div")  # Tahmini etiket
    if term_container:
        for link in term_container.find_all("a"):
            term = link.text.strip()
            if term and len(term.split()) > 0:
                terms.append(term)

terms = list(dict.fromkeys(terms))  # Tekrarları kaldır
df = pd.DataFrame(terms, columns=["Term"])
df.to_csv("odeabank_terms_a_only.csv", index=False)

print(f"{len(terms)} terim çekildi. Örnek: {terms[:5]}")

35 terim çekildi. Örnek: ['Hakkımızda', 'Banka Bilgileri', 'Kurumsal Yönetim', 'Kurumsal Kütüphane', 'İnsan Kaynakları - Odealist']


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.odeabank.com.tr/hakkimizda/oblog/adan-zye-bankacilik-ve-finans-terimleri-sozlugu"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.4 Safari/605.1.15",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9",
    "Accept-Language": "tr-TR,tr;q=0.9,en;q=0.8"
}

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.content, "html.parser")

terms = []

# Tüm <h3> etiketlerini al
for h3 in soup.find_all("h3"):
    term = h3.get_text(strip=True)

    # Sadece "A" harfiyle başlayanlar
    if term and term[0].upper() == "A":
        # Açıklamayı al (bir sonraki <p> elementi)
        p = h3.find_next_sibling("p")
        explanation = p.get_text(strip=True) if p else ""

        terms.append({"Terim": term, "Açıklama": explanation})

# DataFrame'e aktar ve CSV'ye yaz
df = pd.DataFrame(terms)
df.to_csv("odeabank_terimler_A.csv", index=False)

print(df.head())



         Terim                                           Açıklama
0     Arbitraj  Arbitraj, fiyat farkını nakde çevirmek için bi...
1  Açığa Satış  Açığa satış, sahip olunmayan hisse senetlerini...
2   Amortisman  Amortisman, piyasa koşulları veya aşınma ve yı...
3    Akademi’O  Akademi’O; yatırımlarına yön vermek isteyen Od...


### Web scraping

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://www.economist.com/economics-a-to-z/a"

"""
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
#Bot engellerini aşmak veya siteye kimlik bilgisi göndermek için.
"""

try:
    r = requests.get(URL, timeout=10)
    r.raise_for_status() #r.raise_for_status(): HTTP durum kodunu kontrol eder. 200 başarı, 403 erişim engeli, 404 sayfa bulunamadı anlamına gelir.
    print("Sayfa başarıyla alındı, durum kodu:", r.status_code)

    soup = BeautifulSoup(r.content, 'html5lib') #html sayfasını parsel etme

    # dt ve dd etiketlerini bul (svelte-188bkwd sınıfı ile)
    dt_elements = soup.find_all('dt', class_='term-item svelte-188kkwd') #İlgili Verileri Bulma
    dd_elements = soup.find_all('dd', class_='term-item svelte-188kkwd')

    if not dt_elements or not dd_elements:
        print("Hata: 'svelte-188bkwd' sınıfına sahip dt veya dd bulunamadı. HTML yapısını kontrol edin.")
    else:
        # Terim ve açıklamaları eşleştir (sayının eşit olduğunu varsayıyoruz)
        dtt = []
        for dt, dd in zip(dt_elements, dd_elements): #İki listedeki öğeleri eşleştirir (her dt için bir dd)
            term = dt.text.strip() if dt.text else 'Başlık Yok'
            desc = dd.text.strip() if dd.text else 'Açıklama Yok'
            dtt.append((term, desc))
        df = pd.DataFrame(dtt, columns=['terms', 'descriptions'])
        print(df)
except requests.exceptions.RequestException as e:
    print("Hata oluştu:", e)
except Exception as e:
    print("Beklenmeyen bir hata:", e)

Sayfa başarıyla alındı, durum kodu: 200
                    terms                                       descriptions
0      Absolute advantage  A concept that helps to explain international ...
1       Active management  A branch of investment management that attempt...
2      Activist investing  Fund managers who take a stake in a company an...
3       Adverse selection  A risk associated with insurance, and linked t...
4            Agency costs  The expense involved in using a third party to...
..                    ...                                                ...
502           Yield curve  A graph that shows the yield of securities wit...
503      Zero coupon bond  A bond on which no interest payments are made....
504      Zero lower bound  Central banks adjust interest rates in their a...
505  Zero-hours contracts  A form of employment in the gig economy that o...
506         Zero-sum game  The view that economic advance by one party ca...

[507 rows x 2 columns]


In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://www.anadolubank.com.tr/terimler-sozlugu"

"""
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
#Bot engellerini aşmak veya siteye kimlik bilgisi göndermek için.
"""

try:
    r = requests.get(URL, timeout=10)
    r.raise_for_status() #r.raise_for_status(): HTTP durum kodunu kontrol eder. 200 başarı, 403 erişim engeli, 404 sayfa bulunamadı anlamına gelir.
    print("Sayfa başarıyla alındı, durum kodu:", r.status_code)

    soup = BeautifulSoup(r.content, 'html5lib') #html sayfasını parsel etme

    dt_elements = soup.find_all('div', class_="dictionary_area") #İlgili Verileri Bulma

    if not dt_elements:
        print("Hata: 'svelte-188bkwd' sınıfına sahip dt veya dd bulunamadı. HTML yapısını kontrol edin.")
    else:
        # Terim ve açıklamaları eşleştir (sayının eşit olduğunu varsayıyoruz)
        #df = pd.DataFrame(dt_elements)
        print(dt_elements)
except requests.exceptions.RequestException as e:
    print("Hata oluştu:", e)
except Exception as e:
    print("Beklenmeyen bir hata:", e)

Streaming output truncated to the last 5000 lines.
























<h3>Bulls Power</h3>



























<p> Boğa piyasasının gücünü ölçen bir teknik gösterge.</p>
                        </div>, <div class="dictionary_area d-none" data-alphabet="C" data-dic-type="yatirimci">
                            <h3>C2C</h3>

<p> Customer to customer yani tüketiciden tüketiciye anlamına gelen İngilizce terimin kısaltılmışı. Tüketicilerin birbirleri ile gerçekleştirdikleri online (çevrimiçi) alışverişi ifade eden C2C (customer to customer) iş modelinde hem alıcı hem de satıcı tüketicilerdir. </p>



















<h3>Cable</h3>



















<p> GBP/USD paritesinin jargondaki takma ismi.</p>





















<h3>Candlestick Chart</h3>



















<p> Mum çubuğu grafiği. Grafik gösterim türlerinden biri olup teknik analiz araçlarının uygulanmasında en çok kullanılan gösterim şeklidir.</p>





















<h3>Cari Açık</h3>



















<p> Bir ülkede